In [115]:
import os
from typing import Any, Type
from langchain.utilities import WikipediaAPIWrapper
from langchain.document_loaders import WebBaseLoader
from langchain.tools import DuckDuckGoSearchResults
from langchain.tools import WikipediaQueryRun


def wikipedia_search(inputs):
  query = inputs["query"]
  wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=5))
  return wiki.run(query)

def duckduckgo_search(inputs):
  query = inputs["query"]
  search = DuckDuckGoSearchResults()
  return search.run(query)

def web_scraping(inputs):
  url = inputs["url"]
  loader = WebBaseLoader([url])
  docs = loader.load()
  text = "\n\n".join([doc.page_content for doc in docs])
  return text

def save_to_text(inputs):
  filename = inputs["filename"]
  text = inputs["text"]
  os.makedirs("./assistants_research_results", exist_ok=True)
  if not filename.endswith('.txt'):
      filename += '.txt'
  file_path = os.path.join("./assistants_research_results", filename)
  with open(file_path, 'w', encoding='utf-8') as f:
      f.write(text)
  return f"Research results saved to {file_path}"  

functions_map = {
    "wikipedia_search": wikipedia_search,
    "duckduckgo_search": duckduckgo_search,
    "web_scraping": web_scraping,
    "save_to_text": save_to_text,
}

functions = [
  {
    "type": "function",
      "function": {
          "name": "wikipedia_search",
          "description": "Use this tool to perform searches on Wikipedia. It takes a query as an argument. Example query: 'Artificial Intelligence'",
          "parameters": {
              "type": "object",
              "properties": {
                  "query": {
                      "type": "string",
                      "description": "The query you will search for on Wikipedia",
                  }
              },
              "required": ["query"],
          },
      },

  },
  {
      "type": "function",
      "function": {
          "name": "duckduckgo_search",
          "description": "Use this tool to perform web searches using the DuckDuckGo search engine. It takes a query as an argument. Example query: 'Latest technology news'",
          "parameters": {
              "type": "object",
              "properties": {
                  "query": {
                      "type": "string",
                      "description": "The query you will search for",
                  }
              },
              "required": ["query"],
          },
      },
  },
  {
    "type": "function",
      "function": {
          "name": "web_scraping",
          "description": "If you found the website link in DuckDuckGo, Use this to get the content of the link for my research.",
          "parameters": {
              "type": "object",
              "properties": {
                  "url": {
                      "type": "string",
                      "description": "The URL of the website you want to scrape",
                  }
              },
              "required": ["url"],
          },
      },

  },
  {
      "type": "function",
      "function": {
          "name": "save_to_text",
          "description": "Use this tool to save the content as a .txt file.",
          "parameters": {
              "type": "object",
              "properties": {
                  "filename": {
                      "type": "string",
                      "description": "a name of the file you will save the research results",
                  },
                  "text": {
                      "type": "string",
                      "description": "The text you will save to a file.",
                  }
              },
              "required": ["text"],
          },
      },

  }
]

In [116]:
from openai import OpenAI
client = OpenAI()
assistant_id = "asst_Non2tRgu3rvBrRrnS8qHGRny"

In [117]:
thread = client.beta.threads.create(
  messages=[
    {
      "role":"user",
      "content": "Research about Trump Tariff on China in 2025",
    }
  ]
)
thread

Thread(id='thread_anmnLZ1VT6qonsQEg1wzsxeo', created_at=1746893986, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [118]:
import json

def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outpus
    )
    

In [119]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_w98BL7AIU9go000WGLm7dwAT', assistant_id='asst_Non2tRgu3rvBrRrnS8qHGRny', cancelled_at=None, completed_at=None, created_at=1746893996, expires_at=1746894596, failed_at=None, incomplete_details=None, instructions='\n        You are a research expert.\n\n        Your task is to use Wikipedia or DuckDuckGo to gather comprehensive and accurate information about the query provided. \n\n        When you find a relevant website through DuckDuckGo, you must scrape the content from that website. Use this scraped content to thoroughly research and formulate a detailed answer to the question. \n\n        Combine information from Wikipedia, DuckDuckGo searches, and any relevant websites you find. Ensure that the final answer is well-organized and detailed, and include citations with links (URLs) for all sources used.\n\n        Your research should be saved to a .txt file, and the content should match the detailed findings provided. Make sure to include all sources and relevant informat

In [124]:
get_run(run.id, thread.id).status

'completed'

In [97]:
get_run(run.id, thread.id)

Run(id='run_xlGmogR5EGMUZdFCGMil7wke', assistant_id='asst_Non2tRgu3rvBrRrnS8qHGRny', cancelled_at=None, completed_at=1746893396, created_at=1746893270, expires_at=None, failed_at=None, incomplete_details=None, instructions='\n        You are a research expert.\n\n        Your task is to use Wikipedia or DuckDuckGo to gather comprehensive and accurate information about the query provided. \n\n        When you find a relevant website through DuckDuckGo, you must scrape the content from that website. Use this scraped content to thoroughly research and formulate a detailed answer to the question. \n\n        Combine information from Wikipedia, DuckDuckGo searches, and any relevant websites you find. Ensure that the final answer is well-organized and detailed, and include citations with links (URLs) for all sources used.\n\n        Your research should be saved to a .txt file, and the content should match the detailed findings provided. Make sure to include all sources and relevant informat

In [121]:
get_messages(thread.id)

user: Research about Trump Tariff on China in 2025


In [122]:
submit_tool_outputs(run.id, thread.id)

Calling function: wikipedia_search with arg {"query": "Trump Tariff on China"}
Calling function: duckduckgo_search with arg {"query": "Trump Tariff on China 2025"}


Run(id='run_w98BL7AIU9go000WGLm7dwAT', assistant_id='asst_Non2tRgu3rvBrRrnS8qHGRny', cancelled_at=None, completed_at=None, created_at=1746893996, expires_at=1746894596, failed_at=None, incomplete_details=None, instructions='\n        You are a research expert.\n\n        Your task is to use Wikipedia or DuckDuckGo to gather comprehensive and accurate information about the query provided. \n\n        When you find a relevant website through DuckDuckGo, you must scrape the content from that website. Use this scraped content to thoroughly research and formulate a detailed answer to the question. \n\n        Combine information from Wikipedia, DuckDuckGo searches, and any relevant websites you find. Ensure that the final answer is well-organized and detailed, and include citations with links (URLs) for all sources used.\n\n        Your research should be saved to a .txt file, and the content should match the detailed findings provided. Make sure to include all sources and relevant informat